In [1]:
#1
%matplotlib qt

In [2]:
#2
#import stuffs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import ndimage, signal
import os, fnmatch, glob, re, time
from scipy.io import loadmat, savemat
from matplotlib.colors import BoundaryNorm, ListedColormap
from matplotlib import colors
from matplotlib.widgets import RectangleSelector
import cv2
import cv2 as cv
from PIL import Image
# for ginput
#https://medium.com/@vyshali.enukonda/how-to-get-around-runtimeerror-this-event-loop-is-already-running-3f26f67e762e
import nest_asyncio
nest_asyncio.apply()
class StopExecution(Exception):
    def _render_traceback_(self):
        pass




# Image Slices Viewer


Scroll through 2D image slices of a 3D array. https://matplotlib.org/2.1.2/gallery/animation/image_slices_viewer.html



In [6]:
#3
from __future__ import print_function


class IndexTracker(object):
    def __init__(self, ax, X, wv, info = pd.DataFrame({"exposure" : [" "], "power" : [" "], "chipname" : [" "]}),\
                 wv_min = 0, wv_max = 1, wv_step = 1, cmin = 0, cmax = 10000):
        self.ax = ax
        self.wv_min = wv_min
        self.wv_max = wv_max
        self.wv_step = wv_step
        self.wv = wv
        
        ax.set_title('{chipname} at {power}% laser power, {exposure} ms exposure time'.format(chipname = info["chipname"].item(),\
                    power = info["power"].item(), exposure = info["exposure"].item()))
        
       
        
        self.X = X
        self.slices, rows, cols = X.shape
        #start with first slice
        self.ind = 0

        self.im = ax.imshow(self.X[self.ind, :,:], cmap = 'gray')
        plt.colorbar(self.im, label = "[a.u.]")#, ax = self.ax)
        self.im.set_clim(vmin = cmin, vmax = cmax)    
        self.update()
        
        
        
        
        #self.rect = RectangleSelector(self.ax, self.line_select_callback,
        #                               drawtype='box', useblit=True,
        #                               button=[1, 3],  # don't use middle button
        #                               minspanx=5, minspany=5,
        #                               spancoords='pixels',
        #                               interactive=True)

        #plt.connect('key_press_event', self.rect)
        
 
        #Create place holder for spectrum and second figure
        #self.spectrum = np.zeros(self.X.shape[0])
        #self.fig2, self.ax2 = plt.subplots(1,1)
        #self.ax2.set_title('{chipname} at {power}% laser power, {exposure} ms exposure time'.format(chipname = info["chipname"].item(),\
        #            power = info["power"].item(), exposure = info["exposure"].item()))
        #self.ax2.plot(range(self.wv_min, self.wv_max +1), self.spectrum)
        #self.ax2.set_xlabel("Wavelength[nm]")
        #self.ax2.set_ylabel("Signal [a.u.]")
        
        #Create place holder for spectrum and second figure
        #get currArray
        self.currArray = self.X[self.ind, 0:900, 0:300 ]
        #get coordinates at top left
        self.x = 0
        self.y = 0
        self.fig3, self.ax3 = plt.subplots(1,1)
        self.ax3.im = self.ax3.imshow(self.X[self.ind, 0:900, 0:300 ], cmap = 'gray')
        plt.colorbar(self.ax3.im, label = "[a.u.]")
        self.ax3.im.set_clim(vmin = cmin, vmax = cmax)    
        self.update()
        

        def onclick(event):
            coord =  plt.ginput()
            print(coord)
            start_x = int(coord[0][0])-150
            end_x = int(coord[0][0])+150
            
            self.x = int(coord[0][0])-150
            self.y = int(coord[0][1])-450

            start_y = int(coord[0][1])-450
            end_y = int(coord[0][1])+450
            if start_x < 0: 
                start_x = 0
                self.x = 0
            if end_x > 2048:
                end_x = 2047
            if start_y < 0: 
                start_y = 0
                self.y = 0
            if end_y > 2048:
                end_y = 2047
            
            self.ax3.im.set_data(self.X[self.ind,start_y:end_y,\
                                    start_x:end_x])
            self.ax3.im.axes.figure.canvas.draw()
            #savemat("./currColumn.mat", {"Column" : self.X[self.ind,:,int(coord[0][0])-150:int(coord[0][0])+150],\
            #                            "x" : coord[0][0], "y" : coord[0][1]})
            self.currArray = self.X[self.ind,start_y:end_y,\
                                    start_x:end_x]
            
        fig.canvas.mpl_connect('button_press_event', onclick)


        

    def onscroll(self, event):
        #print("%s %s" % (event.button, event.step))
        if event.button == 'up':
            self.ind = (self.ind + 1) % self.slices
        else:
            self.ind = (self.ind - 1) % self.slices
        self.update()

    def update(self):
        self.im.set_data(self.X[self.ind, :, :])
        ax.set_ylabel('Wavelength {wv:.2f} nm'.format(wv = self.wv[self.ind]))
        self.im.axes.figure.canvas.draw()
        
        
    #def line_select_callback(self, eclick, erelease):
    #    #'eclick and erelease are the press and release events'
    #    #x and y are interchanged, because the first dimension is the rows, therefore in representation it becomes the y
    #    self.y1, self.x1 = eclick.xdata, eclick.ydata
    #    self.y2, self.x2 = erelease.xdata, erelease.ydata
        
    #    print("Click Coordinate (%3.2f, %3.2f) --> Release Coordinate(%3.2f, %3.2f)" % (eclick.xdata, eclick.ydata, erelease.xdata, erelease.ydata))
    #    print("The average intensity is %3.2f" % (np.nanmean(self.X[self.ind, \
    #                                                            int(self.x1):int(self.x2),
    #                                                            int(self.y1):int(self.y2)])))
    #    self.spectrum = np.array([np.nanmean(self.X[i, int(self.x1):int(self.x2), int(self.y1):int(self.y2)]) for i in range(self.X.shape[0])])
    #    self.idx = np.where(self.spectrum==self.spectrum.max())[0]
    #    self.ax2.plot(range(self.wv_min, self.wv_max +1), self.spectrum, label = "Peak at %.2f nm \n [%.0f:%.0f, %.0f:%.0f]" % (self.wv[self.ind],self.x1, self.x2, self.y1, self.y2))
    #    self.ax2.legend()
    #    self.fig2.canvas.draw()
    #    #print(" The button you used were: %s %s" % (eclick.button, erelease.button))

        
        

# Load Data

In [4]:
#4
#Load Before Datacube
folder_name = "18"
start_wavelength = 651
end_wavelength = 660

spectra = loadmat("./CS766ProjectDataset/{folder}/Before/{start_wv}.mat".format(start_wv = start_wavelength, folder = folder_name))
spectra = np.array(spectra['spectra'])
data_cube = [spectra]
for i in range(start_wavelength + 1,end_wavelength +1):
    #placeholder
    a = loadmat("./CS766ProjectDataset/{folder}/Before/{start_wv}.mat".format(start_wv = i, folder = folder_name))
    data_cube.append(np.array(a['spectra']))
#convert to np array
before = np.array(data_cube)

#load After Datacube

spectra = loadmat("./CS766ProjectDataset/{folder}/After/{start_wv}.mat".format(start_wv = start_wavelength, folder = folder_name))
spectra = np.array(spectra['spectra'])
data_cube = [spectra]
for i in range(start_wavelength + 1,end_wavelength +1):
    #placeholder
    a = loadmat("./CS766ProjectDataset/{folder}/After/{start_wv}.mat".format(start_wv = i, folder = folder_name))
    data_cube.append(np.array(a['spectra']))
#convert to np array
after = np.array(data_cube)
wavelength = 2 #Technically 650+wavelength, but this gets the correct position in the image stack
beforeSingleWavelength = before[wavelength]
afterSingleWavelength = after[wavelength]


# Visualize Data

In [7]:
#5
#PICK POINT FOR SET OF THREE IN FIRST IMAGE
#Hypercube in the form of an np.array
data = after
#Dimensions of the np.array hypercube
data_shape = before.shape
#wv is an array of wavelengths, incremented 1 at a time
wv = [i for i in range(start_wavelength,end_wavelength+1)]
#??
fig, ax = plt.subplots(1, 1)

#IndexTracker does what?
tracker = IndexTracker(ax, np.resize(data[np.isfinite(data)], data_shape), wv,  \
wv_min = start_wavelength, wv_max = end_wavelength, wv_step = 1,\
cmin = 0, cmax = 10)#3 * np.mean(data[np.isfinite(data)]))


fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()

Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(64.88528138528068, 1032.3658008658006)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(369.6471861471855, 1060.0714285714282)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(59.344155844155125, 1032.3658008658006)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(59.344155844155125, 1048.989177489177)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(26.09740259740181, 1015.742424242424)]


In [8]:
#I separated them again, because otherwise the variables are saved before the user choose any point
#get current matrix data
currArray = tracker.currArray
#top corner x-coordinate
x_top = tracker.x
#top corner y-coordinate
y_top = tracker.y

In [10]:
y_top

565

In [ ]:
#visualize current rectangular matrix image
plt.figure()
plt.imshow(currArray, cmap = 'gray')
plt.clim(0,10)

In [11]:
#6
#PICK POINT FOR SET OF THREE IN FIRST IMAGE
#Hypercube in the form of an np.array of before image
data = before
#Dimensions of the np.array hypercube
data_shape = before.shape
#wv is an array of wavelengths, incremented 1 at a time
wv = [i for i in range(start_wavelength,end_wavelength+1)]
#??
fig, ax = plt.subplots(1, 1)

#IndexTracker does what?
tracker2 = IndexTracker(ax, np.resize(data[np.isfinite(data)], data_shape), wv,  \
wv_min = start_wavelength, wv_max = end_wavelength, wv_step = 1,\
cmin = 0, cmax = 10)#3 * np.mean(data[np.isfinite(data)]))

fig.canvas.mpl_connect('scroll_event', tracker.onscroll)
plt.show()


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(1943.3268398268397, 1015.742424242424)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(1982.1147186147182, 1004.6601731601729)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(2009.8203463203458, 1010.2012987012984)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(2009.8203463203458, 1652.9718614718613)]


Traceback (most recent call last):
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\cbook\__init__.py", line 224, in process
    func(*args, **kwargs)
  File "<ipython-input-6-72976a0944d7>", line 65, in onclick
    coord =  plt.ginput()
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\pyplot.py", line 2383, in ginput
    return gcf().ginput(
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\figure.py", line 2453, in ginput
    return blocking_mouse_input(n=n, timeout=timeout,
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 267, in __call__
    BlockingInput.__call__(self, n=n, timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\blocking_input.py", line 93, in __call__
    self.fig.canvas.start_event_loop(timeout=timeout)
  File "C:\Users\wadi2\Anaconda3\envs\DataCube\lib\site-packages\matplotlib\backends\backend_q

[(1982.1147186147182, 1990.9805194805194)]


In [ ]:
#I separated them again, because otherwise the variables are saved before the user choose any point
#get current matrix data for before
currArray2 = tracker2.currArray
#top corner x-coordinate
x_top2 = tracker2.x
#top corner y-coordinate
y_top2 = tracker2.y

In [ ]:
#visualize rectangular image for before
plt.figure()
plt.imshow(currArray2, cmap = 'gray')
plt.clim(0,10)

In [ ]:
#7

#Load wavelengths 0 is 650
wavelength = 2
beforeSingleWavelength = before[wavelength]
afterSingleWavelength = after[wavelength]



#Remove outliers from both sub-images. If not removed, they cause normalization to fail
Q1 = np.percentile(currArray, 25,interpolation = 'midpoint')

Q3 = np.percentile(currArray, 75, interpolation = 'midpoint')
IQR = Q3 - Q1
 
#Removing the Outliers
currArray[currArray > Q3+1.5*IQR] = Q3+1.5*IQR
currArray[currArray < Q1-1.5*IQR] = Q1-1.5*IQR
#####
Q1 = np.percentile(currArray2, 25,interpolation = 'midpoint')
Q3 = np.percentile(currArray2, 75, interpolation = 'midpoint')
IQR = Q3 - Q1
 
#Removing the Outliers
currArray2[currArray2 > Q3+1.5*IQR] = Q3+1.5*IQR
currArray2[currArray2 < Q1-1.5*IQR] = Q1-1.5*IQR





#similarly for the images
Q1 = np.percentile(beforeSingleWavelength, 25,interpolation = 'midpoint')

Q3 = np.percentile(beforeSingleWavelength, 75, interpolation = 'midpoint')
IQR = Q3 - Q1
 
#Removing the Outliers
beforeSingleWavelength[beforeSingleWavelength > Q3+1.5*IQR] = Q3+1.5*IQR
beforeSingleWavelength[beforeSingleWavelength < Q1-1.5*IQR] = Q1-1.5*IQR
#####
Q1 = np.percentile(afterSingleWavelength, 25,interpolation = 'midpoint')
Q3 = np.percentile(afterSingleWavelength, 75, interpolation = 'midpoint')
IQR = Q3 - Q1
 
#Removing the Outliers
afterSingleWavelength[afterSingleWavelength > Q3+1.5*IQR] = Q3+1.5*IQR
afterSingleWavelength[afterSingleWavelength < Q1-1.5*IQR] = Q1-1.5*IQR



In [ ]:
#8
#Create Hough Circles in both sub-images and save centroid coordinates

#cv.imshow('detected circles',cimg)
dst	= cv.normalize(currArray, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

cimg = cv.cvtColor(dst,cv.COLOR_GRAY2BGR)
dst = cv.medianBlur(dst, 9)
dts = cv.GaussianBlur( dst, (9, 9), 2 )
#cv2.imshow('before',dst)
#Hough circle transform https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga47849c3be0d0406ad3ca45db65a25d2d
# One of the most important features is param1. Min and max raidus will need to be determined based on batch varaition

circles = cv.HoughCircles(dts,cv.HOUGH_GRADIENT,1,250,param1=18,param2=15,minRadius=70,maxRadius=110)

#print(circles)

#show blurred image
if circles is not None:
    circles = np.uint16(np.around(circles))

    for i in circles[0,:]:
        # draw the outer circle
        cv.circle(cimg,(i[0],i[1]),i[2],(0,255,0),3)
        # draw the center of the circle
        cv.circle(cimg,(i[0],i[1]),2,(0,0,255),3)
    #Show before image with circles drown on
    cv2.namedWindow("Before", cv2.WINDOW_NORMAL)
    cv.imshow('Before',cimg)
    #print(circles)
else:
    print("HOUGH CIRCLE DETECTION FAILED ON BEFORE IMAGE")
    #cv.imshow('detected circles',cimg)


###########################################################################
#cv.imshow('detected circles',cimg)
dsta= cv.normalize(currArray2, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
cimga = cv.cvtColor(dsta,cv.COLOR_GRAY2BGR)
dsta = cv.medianBlur(dsta, 9)
dtsa = cv.GaussianBlur( dsta, (9, 9), 2 )
#cv2.imshow('after',dsta)
#Hough circle transform https://docs.opencv.org/4.x/dd/d1a/group__imgproc__feature.html#ga47849c3be0d0406ad3ca45db65a25d2d
# One of the most important features is param1. Min and max raidus will need to be determined based on batch varaition

circlesa = cv.HoughCircles(dtsa,cv.HOUGH_GRADIENT,1,250,param1=18,param2=20,minRadius=70,maxRadius=110)

beforeCircles = []
afterCircles = []
if circlesa is not None:
    circlesa = np.uint16(np.around(circlesa))

    for i in circlesa[0,:]:
        # draw the outer circle
        cv.circle(cimga,(i[0],i[1]),i[2],(0,255,0),3)
        # draw the center of the circle
        cv.circle(cimga,(i[0],i[1]),2,(0,0,255),3)
        
    #Show after image with circles drown on
    cv2.namedWindow("After", cv2.WINDOW_NORMAL)
    cv.imshow('After',cimga)
    #print(circlesa)
else:
    print("HOUGH CIRCLE DETECTION FAILED ON AFTER IMAGE")    
    raise StopExecution
    #Find only a certain number of circles within the images, so that those can be passed to the next step
for i in range(0,len(circles[0])):
    circles[0][i][0] = circles[0][i][0]+x_top
    circles[0][i][1] = circles[0][i][1]+y_top
for i in range(0,len(circlesa[0])):
    circlesa[0][i][0] = circlesa[0][i][0] +x_top2
    circlesa[0][i][1] = circlesa[0][i][1] +y_top2
##TO FIX, DOWNSTREAM NAMING
afterCircles = circlesa
#remove one dimension
afterCircles = afterCircles[0]
afterCircles = afterCircles[afterCircles[:,2].argsort()]
#sort
#afterCircles = np.sort(afterCircles, axis = 1)
beforeCircles = circles
#remove one dimension
beforeCircles = beforeCircles[0]
beforeCircles = beforeCircles[beforeCircles[:,2].argsort()]

#beforeCircles = np.sort(beforeCircles, axis = 1)

#print(afterCircles)
#print(beforeCircles)
            

In [ ]:
#9
##Transform images named
#beforeSingleWavelength
#afterSingleWavelength
#which were defined in the cell above. Use the coordinates stored in
#beforeCircles
#afterCircles
#as [x y radius]

# get difference in x locations between before and after
x_diff = np.mean(np.double(beforeCircles[:,0]) - np.double(afterCircles[:,0]))
y_diff = np.mean(np.double(beforeCircles[:,1]) - np.double(afterCircles[:,1]))

print(np.round(x_diff), np.round(y_diff))
#if difference in x is negative, it means that before is to the left of after. 
#in this case add zero pading to the left of the before image, and add zero pading to the right of after image to ensure
# same matrix sizes
# similarly with y

if x_diff < 0:
    
    beforeSingleWavelength = np.pad(beforeSingleWavelength, ((0,0), (0, np.abs(np.round(x_diff.astype(int))))), constant_values = 0)
    afterSingleWavelength = np.pad(afterSingleWavelength, ((0,0), (np.abs(np.round(x_diff.astype(int))),0)), constant_values = 0)

if y_diff < 0:
    beforeSingleWavelength = np.pad(beforeSingleWavelength, ((0, np.abs(np.round(y_diff.astype(int)))), (0, 0)), constant_values = 0)
    afterSingleWavelength = np.pad(afterSingleWavelength, ((np.abs(np.round(y_diff.astype(int))),0), (0,0)), constant_values = 0)


if x_diff > 0:
    beforeSingleWavelength = np.pad(beforeSingleWavelength, ((0,0), (np.abs(np.round(x_diff.astype(int))), 0)), constant_values = 0)
    afterSingleWavelength = np.pad(afterSingleWavelength, ((0,0), (0, np.abs(np.round(x_diff.astype(int))))), constant_values = 0)

if y_diff > 0:
    beforeSingleWavelength = np.pad(beforeSingleWavelength, ((np.abs(np.round(y_diff.astype(int))), 0), (0, 0)), constant_values = 0)
    afterSingleWavelength = np.pad(afterSingleWavelength, ((0, np.abs(np.round(y_diff.astype(int)))), (0,0)), constant_values = 0)
    



In [ ]:
plt.figure
plt.imshow(np.divide(beforeSingleWavelength, np.max(beforeSingleWavelength)) - np.divide(afterSingleWavelength, np.max(afterSingleWavelength)), cmap = 'gray')
plt.clim(-0.5,0.1)


In [ ]:
plt.figure
plt.imshow(np.divide(beforeSingleWavelength, np.max(beforeSingleWavelength)), cmap = 'gray')
plt.clim(0,1)